In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
!pip install ml-dtypes~=0.2.0
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Prepare data
train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

train_df = pd.read_csv(train_file_path, sep="\t", header=None, names=["type", "msg"]).dropna()
test_df = pd.read_csv(test_file_path, sep="\t", header=None, names=["type", "msg"]).dropna()



In [ ]:
# Prepare labels for training
train_df["type"] = pd.factorize(train_df["type"])[0]
test_df["type"] = pd.factorize(test_df["type"])[0]

train_labels = train_df["type"].values
test_labels = test_df["type"].values

train_ds = tf.data.Dataset.from_tensor_slices((train_df["msg"].values, train_labels))
test_ds = tf.data.Dataset.from_tensor_slices((test_df["msg"].values, test_labels))



In [ ]:
# Constants
BUFFER_SIZE = 100
BATCH_SIZE = 32

train_ds = train_ds.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


In [ ]:
# Tokenize the text data
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=1000, oov_token="<OOV>")
tokenizer.fit_on_texts(train_df["msg"])

train_sequences = tokenizer.texts_to_sequences(train_df["msg"])
test_sequences = tokenizer.texts_to_sequences(test_df["msg"])

train_padded = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=1000, truncating="post", padding="post")
test_padded = tf.keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=1000, truncating="post", padding="post")



In [ ]:
# Create the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=64, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1)
])


In [ ]:
# Compile the model
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(1e-4),
    metrics=['accuracy'],
)

In [ ]:
# Train the model against our data sets
history = model.fit(train_padded, train_labels, validation_data=(test_padded, test_labels), epochs=10, batch_size=BATCH_SIZE)



In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(test_padded, test_labels)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)



In [ ]:
# Create a helper function to plot
def plot_graphs(h, metric):
    plt.plot(h.history[metric])
    plt.plot(h.history['val_'+metric])
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend([metric, 'val_'+metric])

In [ ]:
# Plot the graphs
plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plot_graphs(history, 'accuracy')
plt.ylim(None, 1)
plt.subplot(1, 2, 2)
plot_graphs(history, 'loss')
plt.ylim(0, None)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
# Function to predict messages based on the model
def predict_message(pred_text):
    pred_text_sequence = tokenizer.texts_to_sequences([pred_text])
    pred_text_padded = tf.keras.preprocessing.sequence.pad_sequences(pred_text_sequence, maxlen=1000, truncating="post", padding="post")
    prediction = model.predict(pred_text_padded)
    p = prediction[0][0]
    return [p, "ham" if p < 0.5 else "spam"]

# Example prediction
pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print(prediction)


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()